# Reconstruction of HaloGEM

In [69]:
from tools import *
import pickle

#### 3. compare draft models from template and metacyc

In [87]:
k = 1
meta_model_ids = [rxn.id for rxn in halo_meta.reactions]
for rxn in halo_iJO_meta.reactions:
    if rxn.id in meta_model_ids:
        rxn1 = halo_meta.reactions.get_by_id(rxn.id)
        print(k,rxn.id,rxn.reaction)
        print(k,rxn1.id,rxn1.reaction)
        print('')
        k += 1

1 DEOXYRIBOSE-P-ALD-RXN DEOXY-RIBOSE-5P_c --> ACETALD_c + GAP_c
1 DEOXYRIBOSE-P-ALD-RXN DEOXY-RIBOSE-5P_c <=> ACETALD_c + GAP_c

2 RXN-6321 5-FORMYL-THF_c + PROTON_c --> 5-10-METHENYL-THF_c + WATER_c
2 RXN-6321 5-10-METHENYL-THF_c + WATER_c --> 5-FORMYL-THF_c + PROTON_c

3 RXN0-5240 D-ALANINE_c + PYRIDOXAL_PHOSPHATE_c --> PYRIDOXAMINE-5P_c + PYRUVATE_c
3 RXN0-5240 D-ALANINE_c + PYRIDOXAL_PHOSPHATE_c <=> PYRIDOXAMINE-5P_c + PYRUVATE_c

4 THREONINE-ALDOLASE-RXN THR_c --> ACETALD_c + GLY_c
4 THREONINE-ALDOLASE-RXN THR_c <=> ACETALD_c + GLY_c

5 GLYOHMETRANS-RXN SER_c + THF_c <=> GLY_c + METHYLENE-THF_c + WATER_c
5 GLYOHMETRANS-RXN SER_c + THF_c <=> GLY_c + METHYLENE-THF_c + WATER_c

6 RXN0-6563 CPD0-2189_c <=> GLYCOLALDEHYDE_c + GLY_c
6 RXN0-6563 CPD0-2189_c <=> GLYCOLALDEHYDE_c + GLY_c

7 RXN-13403 2.0 S-ADENOSYLMETHIONINE_c + UROPORPHYRINOGEN-III_c --> 2.0 ADENOSYL-HOMO-CYS_c + DIHYDROSIROHYDROCHLORIN_c + PROTON_c
7 RXN-13403 2.0 S-ADENOSYLMETHIONINE_c + UROPORPHYRINOGEN-III_c --> 2.0 A

217 GUANINE-DEAMINASE-RXN GUANINE_c + PROTON_c + WATER_c --> AMMONIUM_c + XANTHINE_c

218 DTMPKI-RXN ATP_c + TMP_c <=> ADP_c + TDP_c
218 DTMPKI-RXN ATP_c + TMP_c --> ADP_c + TDP_c

219 PHOSPHOGLUCMUT-RXN GLC-1-P_c <=> D-glucopyranose-6-phosphate_c
219 PHOSPHOGLUCMUT-RXN GLC-1-P_c <=> D-glucopyranose-6-phosphate_c

220 RXN0-5185 CPD-2961_c + WATER_c --> GLUCONATE_c + Pi_c
220 RXN0-5185 CPD-2961_c + WATER_c --> GLUCONATE_c + Pi_c

221 RIBOSYLHOMOCYSTEINASE-RXN CPD-564_c --> DIHYDROXYPENTANEDIONE_c + HOMO-CYS_c
221 RIBOSYLHOMOCYSTEINASE-RXN CPD-564_c --> DIHYDROXYPENTANEDIONE_c + HOMO-CYS_c

222 BADH-RXN BETAINE_ALDEHYDE_c + NAD_c + WATER_c --> BETAINE_c + NADH_c + 2.0 PROTON_c
222 BADH-RXN BETAINE_ALDEHYDE_c + NAD_c + WATER_c <=> BETAINE_c + NADH_c + 2.0 PROTON_c

223 PYRNUTRANSHYDROGEN-RXN NADPH_c + NAD_c --> NADH_c + NADP_c
223 PYRNUTRANSHYDROGEN-RXN NADPH_c + NAD_c <=> NADH_c + NADP_c

224 GMP-SYN-GLUT-RXN ATP_c + GLN_c + WATER_c + XANTHOSINE-5-PHOSPHATE_c --> AMP_c + GLT_c + GMP_c + 

## Curate the reversibility of reactions based on iML1515 model

In [104]:
def load_iML1515_reactions_with_metacyc_ids():
    model = cobra.io.read_sbml_model('../../../ComplementaryData/iML1515.xml')
    
    met_rxns = dict()
    for rxn in model.reactions:
        # ['META:3.6.3.10-RXN', 'META:3.6.3.12-RXN', 'META:TRANS-RXN-2']
        metcyc = rxn.annotation.get('biocyc') 
        if metcyc is None: continue
        elif type(metcyc) is list:
            for item in metcyc: met_rxns[item.replace('META:','')] = rxn
        else: met_rxns[metcyc.replace('META:','')] = rxn
    del model
    return met_rxns

In [105]:
def is_reversible(rxn):
    if rxn.lower_bound <0 and rxn.upper_bound >0: return 1
    else: return -1

In [106]:
def set_bound(model,rxn_id,lb=None,ub=None):
    rxn = model.reactions.get_by_id(rxn_id)
    if lb is not None: rxn.lower_bound = lb
    if ub is not None: rxn.upper_bound = ub

In [107]:
def diff_in_reversibility(halo_model,eco_rxns_met):
    for rxn in halo_model.reactions:
        eco_rxn = eco_rxns_met.get(rxn.id)
        if eco_rxn is None: continue

        if is_reversible(eco_rxn)*is_reversible(rxn)>0: continue

        if is_reversible(eco_rxn)>0 and is_reversible(rxn)<0:
            rxn.lower_bound = -1000
            rxn.upper_bound = 1000
            continue

        print(rxn.id)
        print(rxn.reaction)
        print(eco_rxn.reaction)
        print()

In [108]:
eco_rxns_met =  load_iML1515_reactions_with_metacyc_ids()
print(len(eco_rxns_met))

984


In [109]:
 diff_in_reversibility(halo_meta_c,eco_rxns_met)

3-DEHYDROQUINATE-DEHYDRATASE-RXN
DEHYDROQUINATE_c <=> 3-DEHYDRO-SHIKIMATE_c + WATER_c
3dhq_c --> 3dhsk_c + h2o_c

ACETYLGLUTKIN-RXN
ACETYL-GLU_c + ATP_c <=> ADP_c + N-ACETYL-GLUTAMYL-P_c
acglu_c + atp_c --> acg5p_c + adp_c

TRANS-RXN-42
K+_p + PROTON_c <=> K+_c + PROTON_p
h_p + k_c --> h_c + k_p

TRANS-RXN-242
HOMO-SER_p + PROTON_c <=> HOMO-SER_c + PROTON_p
h_p + hom__L_c --> h_c + hom__L_p

TRANS-RXN-122A
L-ASPARTATE_p + 2 PROTON_p <=> L-ASPARTATE_c + 2 PROTON_c
asp__L_p + 2.0 h_p --> asp__L_c + 2.0 h_c

TRANS-RXN0-0244
PROTON_c + THR_p <=> PROTON_p + THR_c
h_p + thr__L_c --> h_c + thr__L_p

RXN-5076
PROTON_p + XANTHINE_p <=> PROTON_c + XANTHINE_c
h_p + xan_p --> h_c + xan_c

RXN0-4083
NA+_p + SER_p <=> NA+_c + SER_c
na1_p + ser__L_p --> na1_c + ser__L_c

TRANS-RXN-132
PROTON_p + URACIL_p <=> PROTON_c + URACIL_c
h_p + ura_p --> h_c + ura_c

RXN0-6
FE+2_p + PROTON_c <=> FE+2_c + PROTON_p
fe2_c + h_p --> fe2_p + h_c

TRANS-RXN-99
CHOLINE_p + PROTON_p <=> CHOLINE_c + PROTON_c
chol_p + h_

In [110]:
# manually modify the reversibility of above reactions
lb_to_be_zero = ['3-DEHYDROQUINATE-DEHYDRATASE-RXN',
                 'ACETYLGLUTKIN-RXN',
                 'TRANS-RXN-122A',
                 'RXN-5076',
                 'RXN0-4083',
                 'TRANS-RXN-132',
                 'TRANS-RXN-99',
                 'TRANS-RXN-3',
                 'TRANS-RXN-118',
                 'TRANS-RXN-121'
                ]
ub_to_be_zero = ['TRANS-RXN-42',
                 'TRANS-RXN-242',
                 'TRANS-RXN0-0244',
                 'RXN0-6',
                 'TRANS-RXN-40']
for rxn_id in lb_to_be_zero: set_bound(halo_meta_c,rxn_id,lb=0)
for rxn_id in ub_to_be_zero: set_bound(halo_meta_c,rxn_id,ub=0)

In [111]:
 diff_in_reversibility(halo_meta_c,eco_rxns_met)

0.295792 ARG_c + 0.241055 ASN_c + 0.09158 CYS_c + 0.26316 GLN_c + 0.26316 GLT_c + 0.612638 GLY_c + 0.094738 HIS_c + 0.290529 ILE_c + 0.513689 L-ALPHA-ALANINE_c + 0.241055 L-ASPARTATE_c + 0.450531 LEU_c + 0.343161 LYS_c + 0.153686 MET_c + 0.185265 PHE_c + 0.221055 PRO_c + 0.215792 SER_c + 0.253687 THR_c + 0.056843 TRP_c + 0.137896 TYR_c + 0.423162 VAL_c --> protein_c
maximal flux: protein_c 0.0


0.026166 DATP_c + 0.027017 DCTP_c + 0.027017 DGTP_c + 0.026166 TTP_c --> dna_c
maximal flux: dna_c 0.0


0.175 ATP_c + 0.133508 CTP_c + 0.215096 GTP_c + 0.144104 UTP_c --> rna_c
maximal flux: rna_c 0.0


0.013013 AMMONIUM_c + 0.005205 CA+2_c + 0.005205 CL-_c + 2.5e-05 CO+2_c + 7e-06 CPD-3_c + 0.000709 CU+2_c + 0.006715 FE+2_c + 0.007808 FE+3_c + 0.195193 K+_c + 0.008675 MG+2_c + 0.000691 MN+2_c + 0.000323 NI+2_c + 0.004338 SULFATE_c + 0.000341 ZN+2_c --> ions_c
maximal flux: ions_c 0.0


0.063814 CPD-12819_c + 0.075214 CPD-17086_c + 0.019456 KDO2-LIPID-IVA_c --> lipids_c
maximal flux: lipids_c 0.0


## Check if model can produce the metabolites in biomass one by one
Match metabolites based on
* metacyc id
* bigg id

In [123]:
def set_yeast_extraction(model,upper_bound):
    # given the evidence that TD01 strain can growth wihout supplyment of amino acids, 
    # the strain should be able to produce 20 amino acids by itself.
    # Block the uptake of 20 amino acids
    amino_acids = ['CYS_c','L-ASPARTATE_c','GLT_c','PHE_c','GLY_c',
                  'HIS_c','ILE_c','LYS_c','LEU_c','MET_c','ASN_c',
                  'PRO_c','GLN_c','ARG_c','SER_c','THR_c','VAL_c',
                  'TRP_c','L-ALPHA-ALANINE_c','TYR_c','L-LACTATE','CPD-15990', 'ADENINE']
    for met_id in amino_acids:
        exg_rxn = model.reactions.get_by_id('Exchange_'+met_id.replace('_c',''))
        exg_rxn.upper_bound = upper_bound

In [124]:
def check_if_model_can_produce_mets_in_biomass_one_by_one(model):
    book = xlrd.open_workbook('../Results/core_biomass_medium_summary_iML1515.xlsx')
    sheets = ['protein','dna','rna','ions','lipids','others']

    mnx2meta = dict()
    for line in open('../../../ComplementaryData/MNX2metacyc.tsv'):
        cont = line.split()
        mnx2meta[cont[0]] = mnx2meta.get(cont[0],[]) + [cont[1]]

    for name in sheets:
        sh = book.sheet_by_name(name)
        print(name)
        for i in range(sh.nrows-1):
            i += 1
            mnx_id = sh.cell(i,9).value
            bigg_id = sh.cell(i,4).value
            met = find_halo_met_c(bigg_id,mnx_id,mnx2meta,model)
            if met is None: print(bigg_id+'_c',0)
            else:
                # add sink reaction
                rxn_id = 'tmp'
                rxn_sink = cobra.Reaction(rxn_id)
                rxn_sink.add_metabolites({met:-1})
                model.add_reaction(rxn_sink)
                model.objective = 'tmp'
                f = model.optimize().objective_value
                model.remove_reactions([rxn_sink])
                print(met.id,met.name,f)
        print('')

In [125]:
set_yeast_extraction(halo_meta_c,0)
check_if_model_can_produce_mets_in_biomass_one_by_one(halo_meta_c)

protein
L-ALPHA-ALANINE_c L-alanine 0.0
ARG_c L-arginine 0.0
ASN_c L-asparagine 0.0
L-ASPARTATE_c L-aspartate 0.0
CYS_c L-cysteine 0.0
GLN_c L-glutamine 0.0
GLT_c L-glutamate 0.0
GLY_c glycine 0.0
HIS_c L-histidine 0.0
ILE_c L-isoleucine 0.0
LEU_c L-leucine 0.0
LYS_c L-lysine 0.0
MET_c L-methionine 0.0
PHE_c L-phenylalanine 0.0
PRO_c L-proline 0.0
SER_c L-serine 0.0
THR_c L-threonine 0.0
TRP_c L-tryptophan 0.0
TYR_c L-tyrosine 0.0
VAL_c L-valine 0.0

dna
DATP_c dATP 0.0
DCTP_c dCTP 0.0
DGTP_c dGTP 0.0
TTP_c dTTP 0.0

rna
ATP_c ATP 0.0
CTP_c CTP 0.0
GTP_c GTP 0.0
UTP_c UTP 0.0

ions
CO+2_c Co2+ 1000.0
CU+2_c Cu2+ 1000.0
FE+2_c Fe2+ 0.0
FE+3_c Fe3+ 1000.0
K+_c  0.0
MG+2_c Mg2+ 1000.0
MN+2_c Manganese 1000.0
CPD-3_c molybdate 0.0
NI+2_c  0.0
SULFATE_c sulfate 0.0
ZN+2_c  1000.0
CA+2_c  1000.0
CL-_c chloride 1000.0
AMMONIUM_c ammonium 0.0

lipids
KDO2-LIPID-IVA_c alpha-Kdo-(2->4)-alpha-Kdo-(2->6)-lipid IVA (E. coli) 0.0
CPD-12819_c Phosphatidylethanolamine (dihexadecanoyl, n-C16:0) 0.0
CPD

### 9. Gap-filling with iML1515

In [126]:
def do_gap_filling(halo_met,halmodel,universal,**kwargs):
 
    rxn_id = 'tmp'
    try: 
        rxn_sink = halmodel.reactions.get_by_id(rxn_id)
        halmodel.remove_reactions([rxn_sink])
    except: None
    rxn_sink = cobra.Reaction(rxn_id)
    rxn_sink.add_metabolites({met:-1})
    halmodel.add_reaction(rxn_sink)
    halmodel.objective = 'tmp'
    f1 = halmodel.optimize().objective_value
    
    added_genes = dict()# {'rxn_id':'genes'}
    print(halo_met.id)
    print('Before gap-filling:',f1)
    if f1 != 0: print('No need for gap-filling')
    else:
        try: 
            with universal:
                solution = cobra.flux_analysis.gapfill(halmodel, universal,**kwargs)
            all_rxns = dict()
            rxns_to_add = list()
            for rxn in solution[0]:
                try: halmodel.reactions.get_by_id(rxn.id)
                except: 
                    rxn_clean = rxn.copy()
                    added_genes[rxn_clean.id] = rxn_clean.gene_reaction_rule
                    
                    # add _missing tag for those genes
                    new_gr = rxn_clean.gene_reaction_rule
                    for gene in parse_gr(rxn_clean.gene_reaction_rule):
                        new_gr = new_gr.replace(gene,gene+'_missing')
        
                    rxn_clean.gene_reaction_rule = new_gr
                    rxns_to_add.append(rxn_clean)
            halmodel.add_reactions(rxns_to_add)
            f2 = halmodel.optimize().objective_value
            print('After gap-filling:',f2)
            print('Added reactions:')
            for rxn in rxns_to_add:print(rxn.id,rxn.reaction)
        except: print('Gap-filling failed for',halo_met.id)

    halmodel.remove_reactions([rxn_sink])
    halmodel.repair()
    print('\n\n')
    return added_genes

def do_gap_filling_combine(halo_met,halmodel,universal,all_rxns,**kwargs):
    # add filled reactions in the end, not after each step
    rxn_id = 'tmp'
    try: 
        rxn_sink = halmodel.reactions.get_by_id(rxn_id)
        halmodel.remove_reactions([rxn_sink])
    except: None
    rxn_sink = cobra.Reaction(rxn_id)
    rxn_sink.add_metabolites({met:-1})
    halmodel.add_reaction(rxn_sink)
    halmodel.objective = 'tmp'
    f1 = halmodel.optimize().objective_value
    
    added_genes = dict()# {'rxn_id':'genes'}
    print(halo_met.id)
    print('Before gap-filling:',f1)
    if f1 != 0: print('No need for gap-filling')
    else:
        try: 
            with universal:
                solution = cobra.flux_analysis.gapfill(halmodel, universal,**kwargs)
            rxns_to_add = list()
            for rxn in solution[0]:
                try: halmodel.reactions.get_by_id(rxn.id)
                except: 
                    rxn_clean = rxn.copy()
                    added_genes[rxn_clean.id] = rxn_clean.gene_reaction_rule
                    
                    # add _missing tag for those genes
                    new_gr = rxn_clean.gene_reaction_rule
                    for gene in parse_gr(rxn_clean.gene_reaction_rule):
                        new_gr = new_gr.replace(gene,gene+'_missing')
        
                    rxn_clean.gene_reaction_rule = new_gr
                    rxns_to_add.append(rxn_clean)
                    all_rxns[rxn_clean.id] = rxn_clean
                    
            #halmodel.add_reactions(rxns_to_add)
            #f2 = halmodel.optimize().objective_value
            #print('After gap-filling:',f2)
            print('Added reactions:')
            for rxn in rxns_to_add:print(rxn.id,rxn.reaction)
        except: print('Gap-filling failed for',halo_met.id)

    halmodel.remove_reactions([rxn_sink])
    halmodel.repair()
    print('\n\n')
    return added_genes,all_rxns

In [127]:
def try_gap_filling(halo_met,halmodel,universal, **kwargs):
    # glycogen_c
    
    solution = [[]]
    rxn_id = 'tmp'
    try: 
        rxn_sink = halmodel.reactions.get_by_id(rxn_id)
        halmodel.remove_reactions([rxn_sink])
        halomodel.repair()
    except: None
    rxn_sink = cobra.Reaction(rxn_id)
    rxn_sink.add_metabolites({met:-1})
    halmodel.add_reaction(rxn_sink)
    halmodel.objective = 'tmp'
    optres = halmodel.optimize()
    f1 = optres.objective_value
    print('Before gap-filling:',f1)
    if f1 != 0: 
        print('No need for gap-filling')
        solution = [['No need for gap-filling']]
        
    else:
        try: 
            solution = cobra.flux_analysis.gapfill(halmodel, universal,**kwargs)
            halmodel.add_reactions(solution[0])
            f2 = halmodel.optimize().objective_value
            print('After gap-filling:',f2)
            print('Added reactions:')
            for rxn in solution[0]:print(rxn.id,rxn.reaction)
            halmodel.remove_reactions(solution[0])
        except: 
            print('Gap-filling failed for',halo_met.id)
            solution = [['Failed']]
    halmodel.remove_reactions([rxn_sink])
    print('\n\n')
    return solution

#### Block uptake of 20 amino acids,  do gap-filling with iJO1366 as universal model. Do gap-filling for metabolites in order of (1) DNA , RNA, Protein, lipids, Others

In [128]:
fhand = open('../Results/gap_filled_reactions_v1_without_missing_genes.txt','w')

In [129]:
def write_gap_fill_res(fhand,added_genes,met):
    fhand.write('>{0} {1}\n'.format(met.id,met.name))
    if len(added_genes)==0: fhand.write('None')
    else:
        for k,v in added_genes.items(): fhand.write('{0},{1}\n'.format(k,v))
    fhand.write('\n')

In [130]:
set_yeast_extraction(halo_meta_c,0)
# ATP
#met = halo_meta_c.metabolites.ATP_c
#added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False)
#write_gap_fill_res(fhand,added_genes,met)

# DNA
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.dna_c
added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# RNA 
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.rna_c
added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# Protein
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.protein_c
added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# lipids
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.lipids_c
added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)


# Others
book = xlrd.open_workbook('../Results/core_biomass_medium_summary_iML1515.xlsx')
sh = book.sheet_by_name('others')
for i in range(sh.nrows-1):
    set_yeast_extraction(halo_meta_c,0)
    i += 1
    eco_met = eco.metabolites.get_by_id(sh.cell(i,0).value)
    met = create_universal_met(eco_met,halo_meta_c,universal,mnx2meta)
    print(met)
    added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False,iterations=1)
    write_gap_fill_res(fhand,added_genes,met)
    print('')
fhand.close()

dna_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpvregvza4.lp
Reading time = 0.02 seconds
: 2213 rows, 4008 columns, 18884 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp0vprkb4i.lp
Reading time = 0.02 seconds
: 1872 rows, 5380 columns, 20970 nonzeros
After gap-filling: 38.93863862420134
Added reactions:
EX_h_e PROTON_e <=> 
FTHFLi ATP_c + FORMATE_c + THF_c --> 10-FORMYL-THF_c + ADP_c + Pi_c
Htex PROTON_e <=> PROTON_p
GLCtex_copy1 Glucopyranose_e --> Glucopyranose_p
PItex CPD-16459_e <=> Pi_p
FE2tex FE+2_e <=> FE+2_p
UREAtex UREA_e <=> UREA_p
CITtex CIT_e <-- CIT_p
GLUt2rpp GLT_p + PROTON_p <=> GLT_c + PROTON_c
UREAtpp UREA_p <=> UREA_c
CITt3pp CIT_c + PROTON_p --> CIT_p + PROTON_c



rna_c
Before gap-filling: 6.70011100029226
No need for gap-filling



protein_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmptbz0rt_d.lp

Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'Htex' since it already exists.
Ignoring reaction 'FE2tex' since it already exists.
Ignoring reaction 'UREAtex' since it already exists.
Ignoring reaction 'CITtex' since it already exists.
Ignoring reaction 'UREAtpp' since it already exists.
Ignoring reaction 'CITt3pp' since it already exists.


After gap-filling: 6.792163186719584
Added reactions:
ASNS2 AMMONIUM_c + ATP_c + L-ASPARTATE_c --> AMP_c + ASN_c + PPI_c + PROTON_c
HSK ATP_c + HOMO-SER_c --> ADP_c + O-PHOSPHO-L-HOMOSERINE_c + PROTON_c
G5SADs L-GLUTAMATE_GAMMA-SEMIALDEHYDE_c --> L-DELTA1-PYRROLINE_5-CARBOXYLATE_c + PROTON_c + WATER_c
PPND NAD_c + PREPHENATE_c --> CARBON-DIOXIDE_c + NADH_c + P-HYDROXY-PHENYLPYRUVATE_c
ADSK APS_c + ATP_c --> ADP_c + PAPS_c + PROTON_c
HSDy HOMO-SER_c + NADP_c <=> L-ASPARTATE-SEMIALDEHYDE_c + NADPH_c + PROTON_c
SO4tex HSO4_e <=> SULFATE_p
NO2t2rpp NITRITE_p + PROTON_p <=> NITRITE_c + PROTON_c
THDPS DELTA1-PIPERIDEINE-2-6-DICARBOXYLATE_c + SUC-COA_c + WATER_c --> CO-A_c + N-SUCCINYL-2-AMINO-6-KETOPIMELATE_c
GLCtex_copy2 Glucopyranose_e <=> Glucopyranose_p



lipids_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpabn4zewd.lp
Reading time = 0.02 seconds
: 2216 rows, 4050 columns, 19030 nonzeros
Read LP format model from file /var/f

Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'FTHFLi' since it already exists.
Ignoring reaction 'Htex' since it already exists.
Ignoring reaction 'PItex' since it already exists.
Ignoring reaction 'FE2tex' since it already exists.
Ignoring reaction 'UREAtex' since it already exists.
Ignoring reaction 'CITtex' since it already exists.
Ignoring reaction 'UREAtpp' since it already exists.
Ignoring reaction 'CITt3pp' since it already exists.
Ignoring reaction 'GLCtex_copy2' since it already exists.


After gap-filling: 24.59910591105682
Added reactions:
GLUt4pp GLT_p + NA+_p --> GLT_c + NA+_c
PSSA161 SER_c + cdpdhdec9eg_c --> CMP_c + PROTON_c + ps161_c
PSSA160 SER_c + cdpdhdecg_c --> CMP_c + CPD-12817_c + PROTON_c



PROTON_c
PROTON_c
Before gap-filling: 1000.0
No need for gap-filling




10-FORMYL-THF_c
10-FORMYL-THF_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp1pqasgd3.lp
Reading time = 0.02 seconds
: 2216 rows, 4056 columns, 19058 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpfwu1y7a0.lp
Reading time = 0.02 seconds
: 1872 rows, 5380 columns, 20970 nonzeros


...

After gap-filling: 3.7593984962406037
Added reactions:
PPPGO3 3.0 FUM_c + PROTOPORPHYRINOGEN_c --> PROTOPORPHYRIN_IX_c + 3.0 SUC_c




PYRIDOXAL_PHOSPHATE_c
PYRIDOXAL_PHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpxre1qg9z.lp
Reading time = 0.02 seconds
: 2221 rows, 4100 columns, 19228 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpbpwc_vh3.lp
Reading time = 0.02 seconds
: 1872 rows, 5380 columns, 20970 nonzeros


Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'Htex' since it already exists.
Ignoring reaction 'PItex' since it already exists.
Ignoring reaction 'FE2tex' since it already exists.
Ignoring reaction 'UREAtex' since it already exists.
Ignoring reaction 'CITtex' since it already exists.
Ignoring reaction 'UREAtpp' since it already exists.
Ignoring reaction 'CITt3pp' since it already exists.
Ignoring reaction 'GLCtex_copy2' since it already exists.


After gap-filling: 19.230769230769198
Added reactions:
GLYCLTt2rpp GLYCOLLATE_p + PROTON_p <=> GLYCOLLATE_c + PROTON_c
PDX5PO2 NAD_c + PYRIDOXINE-5P_c --> NADH_c + PROTON_c + PYRIDOXAL_PHOSPHATE_c




RIBOFLAVIN_c
RIBOFLAVIN_c
Before gap-filling: 10.526315789473678
No need for gap-filling




SIROHEME_c
SIROHEME_c
Before gap-filling: 4.310344827586206
No need for gap-filling




SUC-COA_c
SUC-COA_c
Before gap-filling: 5.586592178770935
No need for gap-filling




THF_c
THF_c
Before gap-filling: 8.695652173913045
No need for gap-filling




THIAMINE-PYROPHOSPHATE_c
THIAMINE-PYROPHOSPHATE_c
Before gap-filling: 0.0
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmpnp4ohb3m.lp
Reading time = 0.02 seconds
: 2221 rows, 4104 columns, 19246 nonzeros
Read LP format model from file /var/folders/dx/ghtq02dx2w307xx_5hncb421qny3_w/T/tmp65a1lioj.lp
Reading time = 0.02 seconds
: 1872 rows, 5380 columns, 20970 nonzeros


Ignoring reaction 'EX_h_e' since it already exists.
Ignoring reaction 'FTHFLi' since it already exists.
Ignoring reaction 'Htex' since it already exists.
Ignoring reaction 'PItex' since it already exists.
Ignoring reaction 'FE2tex' since it already exists.
Ignoring reaction 'UREAtex' since it already exists.
Ignoring reaction 'CITtex' since it already exists.
Ignoring reaction 'UREAtpp' since it already exists.
Ignoring reaction 'CITt3pp' since it already exists.
Ignoring reaction 'ADSK' since it already exists.
Ignoring reaction 'SO4tex' since it already exists.
Ignoring reaction 'GLCtex_copy2' since it already exists.
Ignoring reaction 'NADH18pp' since it already exists.
Ignoring reaction 'O2tex' since it already exists.


After gap-filling: 7.142857142857134
Added reactions:
O2tpp OXYGEN-MOLECULE_p <=> OXYGEN-MOLECULE_c
THZPSN3 ATP_c + CPD-12279_c + DEOXYXYLULOSE-5P_c + NADPH_c + PROTON_c + iscssh_c --> AMP_c + CARBON-DIOXIDE_c + NADP_c + PPI_c + THZ-P_c + 2.0 WATER_c + iscs_c




CPD-9649_c
CPD-9649_c
Before gap-filling: 2.409638554216865
No need for gap-filling




ADP_c
ADP_c
Before gap-filling: 14.084507042253584
No need for gap-filling




Pi_c
Pi_c
Before gap-filling: 1000.0
No need for gap-filling




PPI_c
PPI_c
Before gap-filling: 124.99999999999997
No need for gap-filling




ATP_c
ATP_c
Before gap-filling: 12.65822784810122
No need for gap-filling






set_yeast_extraction(halo_meta_c,0)
all_filled_rxns = {}
# ATP
#met = halo_meta_c.metabolites.ATP_c
#added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False)
#write_gap_fill_res(fhand,added_genes,met)

# DNA
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.dna_c
added_genes,all_filled_rxns = do_gap_filling_combine(met,halo_meta_c,universal,all_filled_rxns,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# RNA 
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.rna_c
added_genes,all_filled_rxns = do_gap_filling_combine(met,halo_meta_c,universal,all_filled_rxns,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# Protein
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.protein_c
added_genes,all_filled_rxns = do_gap_filling_combine(met,halo_meta_c,universal,all_filled_rxns,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)

# lipids
set_yeast_extraction(halo_meta_c,0)
met = halo_meta_c.metabolites.lipids_c
added_genes,all_filled_rxns = do_gap_filling_combine(met,halo_meta_c,universal,all_filled_rxns,demand_reactions=False,iterations=1)
write_gap_fill_res(fhand,added_genes,met)


# Others
book = xlrd.open_workbook('../Results/core_biomass_medium_summary_iML1515.xlsx')
sh = book.sheet_by_name('others')
for i in range(sh.nrows-1):
    set_yeast_extraction(halo_meta_c,0)
    i += 1
    eco_met = eco.metabolites.get_by_id(sh.cell(i,0).value)
    met = create_universal_met(eco_met,halo_meta_c,universal,mnx2meta)
    print(met)
    added_genes,all_filled_rxns = do_gap_filling_combine(met,halo_meta_c,universal,all_filled_rxns,demand_reactions=False,iterations=1)
    write_gap_fill_res(fhand,added_genes,met)
    print('')
fhand.close()
halo_meta_c.add_reactions(list(all_filled_rxns.values()))

In [131]:
report_model_status(halo_meta_c)

Number of reactions: 2053
Number of metabolits: 2221
Number of compartments: 3
Number of genes: 1377
Number of missing genes: 53
Number of reactions with missing genes: 41


#### Update missing genes added by gap-filling. By find homologues in H. TD01

In [132]:
halo_meta_c = update_missing_genes(halo_meta_c,1e-3,report=True,coverage=45)

Number of reactions: 2053
Number of metabolits: 2220
Number of compartments: 3
Number of genes: 1342
Number of missing genes: 10
Number of reactions with missing genes: 2


#### For those reactions with missing genes, remove its gr rule and put a note as : 
{'source':'gapfilled from iJO1366, genes were not found in H.TD01'}

In [133]:
updated_rxns = list()
for rxn in halo_meta_c.reactions:
    if '_missing' in rxn.gene_reaction_rule:
        print(rxn.id,rxn.gene_reaction_rule)
        rxn.gene_reaction_rule = ''
        rxn.notes['source']='gapfilled from iJO1366, genes were not found in H.TD01'
    updated_rxns.append(rxn)

print('')
halo_meta_c = cobra.Model('model_v1')
halo_meta_c.add_reactions(updated_rxns)
report_model_status(halo_meta_c)

NADH18pp TD01GL000419 and TD01GL002642 and TD01GL002645 and TD01GL002810 and b2279_missing and b2280_missing and b2281_missing and b2282_missing and b2283_missing and b2286_missing and b2287_missing and b2288_missing
THZPSN3 TD01GL000663 and TD01GL001192 and TD01GL003669 and b0423_missing and b3990_missing

Number of reactions: 2053
Number of metabolits: 2220
Number of compartments: 3
Number of genes: 1330
Number of missing genes: 0
Number of reactions with missing genes: 0


book = xlrd.open_workbook('../Results/core_biomass_medium_summary.xlsx')
sh = book.sheet_by_name('protein')
for i in range(sh.nrows-1):
    i += 1
    eco_met = eco.metabolites.get_by_id(sh.cell(i,0).value)
    met = create_universal_met(eco_met,halo_meta_c,universal,mnx2meta)
    print(met)
    added_genes = do_gap_filling(met,halo_meta_c,universal,demand_reactions=False)
    #write_gap_fill_res(fhand,added_genes,met)
    print('')

### Add Biomass verions 1

0.000223 10-FORMYL-THF_c + 0.000223 2-OCTAPRENYL-6-HYDROXYPHENOL_c + 2.6e-05 2fe2s_c + 75.37723 ATP_c + 2e-06 BIOTIN_c + 0.000576 CO-A_c + 0.00026 CPD-7_c + 5.5e-05 CPD-9649_c + 0.013894 CPD0-2278_p + 0.000223 FAD_c + 0.000223 METHYLENE-THF_c + 0.000447 NADP_c + 0.001831 NAD_c + 0.000223 PROTOHEME_c + 0.000223 PYRIDOXAL_PHOSPHATE_c + 0.000223 RIBOFLAVIN_c + 0.000223 S-ADENOSYLMETHIONINE_c + 0.000223 SIROHEME_c + 9.8e-05 SUC-COA_c + 0.000223 THF_c + 0.000223 THIAMINE-PYROPHOSPHATE_c + 70.028756 WATER_c + dna_c + ions_c + lipids_c + protein_c + rna_c --> 75.37723 ADP_c + 0.773903 PPI_c + 75.37723 PROTON_c + 75.37323 Pi_c


In [135]:
halo_meta_c.reactions.Exchange_Glucopyranose.upper_bound = 5
halo_meta_c.reactions.get_by_id('Exchange_L-LACTATE').upper_bound=0
halo_meta_c.reactions.get_by_id('Exchange_CPD-15990').upper_bound=0

In [136]:
#print(halo_meta_c.reactions.Exchange_Glucopyranose.upper_bound)
set_yeast_extraction(halo_meta_c,0)
halo_meta_c.objective = 'Biomass_v1'
print(halo_meta_c.optimize())

<Solution 0.000 at 0x13360997f0>


In [137]:
set_yeast_extraction(halo_meta_c,1000)
halo_meta_c.objective = 'Biomass_v1'
print(halo_meta_c.optimize())

<Solution 0.000 at 0x1336099a20>


#### Save model and filled reactions

In [138]:
set_yeast_extraction(halo_meta_c,0)
check_if_model_can_produce_mets_in_biomass_one_by_one(halo_meta_c)

protein
L-ALPHA-ALANINE_c L-alanine 3.5294117647058822
ARG_c L-arginine 1.0714285714285723
ASN_c L-asparagine 3.333333333333334
L-ASPARTATE_c L-aspartate 8.571428571428573
CYS_c L-cysteine 0.923076923076923
GLN_c L-glutamine 1.8750000000000013
GLT_c L-glutamate 2.857142857142861
GLY_c glycine 20.0
HIS_c L-histidine 1.0526315789473684
ILE_c L-isoleucine 1.0169491525423715
LEU_c L-leucine 1.0169491525423728
LYS_c L-lysine 1.03448275862069
MET_c L-methionine 0.6451612903225806
PHE_c L-phenylalanine 0.5940594059405943
PRO_c L-proline 1.3333333333333286
SER_c L-serine 12.0
THR_c L-threonine 3.1578947368421053
TRP_c L-tryptophan 0.46874999999999956
TYR_c L-tyrosine 0.6741573033707868
VAL_c L-valine 1.3333333333333335

dna
DATP_c dATP 0.4444444444444464
DCTP_c dCTP 0.5405405405405405
DGTP_c dGTP 0.4878048780487809
TTP_c dTTP 0.5263157894736843

rna
ATP_c ATP 0.48780487804878003
CTP_c CTP 0.6060606060606069
GTP_c GTP 0.5405405405405403
UTP_c UTP 0.6818181818181819

ions
CO+2_c Co2+ 1000.0
CU+2

In [139]:
for met in halo_meta_c.metabolites: 
    try: met.charge = int(met.charge)
    except: pass

#### Save_model

In [140]:
import sys
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils

In [141]:
utils.save_model_into_formats(halo_meta_c,'../../../ModelFiles/','Halo_GEM_v1')

ValueError: invalid null reference in method 'Species_setCompartment', argument 2 of type 'std::string const &'

cobra.io.write_sbml_model(halo_meta_c,
        '../../../ModelFiles/xml/combine_halo_meta100_iML1515_gap_fill_with_core_biomass_without_missing_genes.xml')
fhand.close()

cobra.io.save_matlab_model(halo_meta_c,
'../../../ModelFiles/mat/combine_halo_meta100_iML1515_gap_fill_with_core_biomass_without_missing_genes.mat')

cobra.io.save_json_model(halo_meta_c,
        '../../../ModelFiles/json/combine_halo_meta100_iML1515_gap_fill_with_core_biomass_without_missing_genes.json')

cobra.io.save_yaml_model(halo_meta_c,
        '../Results/combine_halo_meta100_iJO1366_gap_fill_with_core_biomass_without_missing_genes.yaml')

In [ ]:
report_model_status(halo_meta_c)

In [ ]:
report_model_status(eco)

#### Compare reactions: model_with/without missing genes

model_with_missing_genes = cobra.io.read_sbml_model('../Results/combine_halo_meta100_iJO1366_gap_fill_with_core_biomass_with_missing_genes.xml')

k = 0
missing_reactions = 0
for rxn in model_with_missing_genes.reactions:
    if '_missing' in rxn.gene_reaction_rule:
        missing_reactions += 1
        try: 
            halo_meta_c.reactions.get_by_id(rxn.id)
            k += 1
        except: None
print('Number of reactions with missing genes that were found by gap-filling:',k,'/',missing_reactions)